20180634_최준혁_NLP_중간과제

## 과제 시작 전 준비

In [1]:
import re, os, nltk #이외의 모듈 사용X

In [2]:
for Folder in ['./Q1_Out','./Q2_Out']:   #1번, 2번 문제 출력 파일 저장을 위한 폴더 만들기
    os.makedirs(Folder, exist_ok = True) # exist_ok = True : 동일 폴더 존재 무시 

# 1번
과제 설명 
- text : 파일을 open 후 read 해서 lower화 시킴
- punctuation : text에서 \W+와 "_"를 이용해 문장부호 추출한 후 하나의 문자열로 ''.join하여 문장부호 문자열 뭉치를 만듦
- word_list : text를 공백단위(\n \t 빈문자열)로 나누기 위해 split()
- indices : 검색단어를 word_list에서 발견 시 앞뒤 인덱스를 튜플쌍으로 저장
    - 참고사항 : 앞뒤 단어를 구할때 인덱스 +1, -1을 구하는 형태라 전체 텍스트 맨 앞이나 뒤 단어를 검색하면 오류날 것으로 예상됨. 하지만 별도 지침이 없어서 예외처리는 하지 않았습니다
- freqD : indices에서 받아온 인덱스로 형식에 맞게 변환 후 먼저 알파벳 순으로 정렬하고 Freqdist하여서 동일 빈도면 알파벳 오름차순이도록 함
- filtered_freqD : 최소 빈도수 제한하기
- 이후 과정 ; Freqdist를 형식에 맞게 문자열화하여 파일 생성 
    - 참고사항 : with open as f 이런 식으로 처리하는 것이 강의자료에 없어서 혹시 몰라 직접 open하여 받고 close하였습니다

In [3]:
# 1번 
def contexts(folder, filename, search_word, atleast):
    text = open(f'./Q1/{filename}','r',encoding ='utf').read().lower()
    punctuation  = ''.join(set(re.findall('[\W]',text)))+'_'
    word_list = [word.strip(punctuation) for word in text.split()]
    indices = [(index-1,index+1) for index in range(len(word_list)) if word_list[index] == search_word.lower()]
    freqD = nltk.FreqDist(sorted([f'{word_list[i]}_{word_list[j]}' for i,j in indices]))
    filtered_freqD = nltk.FreqDist({item: freq for item, freq in freqD.items() if freq >= atleast})
    f = open(f'./Q1_Out/{filename[:-4]}_context_{search_word.lower()}_{atleast}.txt','w',encoding ='utf')
    f.write("\n".join([f'{i}\t{j}' for i,j in filtered_freqD.most_common()]))
    f.close()
contexts('Q1', 'Q1_data02.txt', 'hand', 2)
contexts('Q1', 'Q1_data01.txt', 'The', 3)
contexts('Q1', 'Q1_data01.txt', 'through', 1)
contexts('Q1', 'Q1_data01.txt', 'HAND', 2)
contexts('Q1', 'Q1_data02.txt', 'THE', 7)

# 2번
과제 설명
- fileids : Q2의 모든 파일명 호출
- for 루프 : Q2 모든 파일을 하나씩 불러와서 Q2_Out에 각각의 morph.txt를 생성하기 위한 루프
    - text : Q2의 파일을 read하여 가져옴
    - line_list : 정규표현으로 원하는 정보가 있는 줄을 가져와서 " + "로 join하고 다시 " + "로 split하여 한개의 형태소 분석이 각각 리스트의 원소가 되도록 저장
    - 파일 저장 : morph.txt를 생성하고 line_list 각 원소를 \n를 붙여서 line by line으로 write

In [4]:
# 2번
fileids = nltk.corpus.PlaintextCorpusReader('./Q2','.*').fileids()
for k in range(len(fileids)):
    text = open(f'./Q2/{fileids[k]}','r',encoding ='utf').read()
    line_list = " + ".join(re.findall('[\w-]+\t.+\t(.+)',text)).split(" + ")
    f = open(f'./Q2_Out/{fileids[k][:-4]}_morph.txt','w',encoding ='utf')
    for line in line_list: f.write(f'{line}\n')
    f.close()

# 3번
과제 설명
- fileids : Q3폴더의 파일명들
- cfd_input : Q3폴더의 모든 파일을 확인하여 (장르, 문장부호) 튜플원소로 이루어진 리스트 생성
- 그것을 ConditionalFreqDist로 만들고 tabulate()하여 보여줌.

In [5]:
# 3번
fileids = nltk.corpus.PlaintextCorpusReader('./Q3','.*').fileids()
cfd_input = [(fileids[i][2], w) for i in range(len(fileids)) for w in re.findall("(.+)/SP",\
            open(f'./Q3/{fileids[i]}','r',encoding ='utf').read())]
nltk.ConditionalFreqDist(cfd_input).tabulate()

      ,     .     /     :     ;     ·     ，     ：     ； 
A  6708  3673  2925    91    68  2585     0     0     0 
B  4171  1047   367   204     3   610     0     5     0 
D  2429    55    10    97     5     0     0     0     0 
E  8567    12     5    45     0    56     0     0     0 
G  3125     9     0    12     0    90     0     0     0 
H 35348  2293   214  1136    94  4514     1     1     4 
J   335     1     2     0     0     0     0     0     0 


Appendix : 정답 확인용 코드

In [6]:
def compare_text_files(file1_path, file2_path):
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
        content1 = file1.read()
        content2 = file2.read()
        
        if content1 == content2:
            return True
        else:
            print("error",file1_path)
            return False

In [7]:
# 1번 정답
Ans = nltk.corpus.PlaintextCorpusReader('./Q1_Ans','.*').fileids()
Out = nltk.corpus.PlaintextCorpusReader('./Q1_Out','.*').fileids()
for i in range(len(Out)):
    print(i+1, compare_text_files(f'./Q1_Ans/{Ans[i]}', f'./Q1_Out/{Out[i]}'))

1 True
2 True
3 True
4 True
5 True


In [8]:
# 2번 정답
Ans = nltk.corpus.PlaintextCorpusReader('./Q2_Ans','.*').fileids()
Out = nltk.corpus.PlaintextCorpusReader('./Q2_Out','.*').fileids()
for i in range(len(Out)):
    print(i+1, compare_text_files(f'./Q2_Ans/{Ans[i]}', f'./Q2_Out/{Out[i]}'))

1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 True
22 True
23 True
24 True
25 True
26 True
27 True
28 True
29 True
30 True
31 True
32 True
33 True
34 True
35 True
36 True
37 True
38 True
39 True
40 True
41 True
42 True
43 True
44 True
45 True
46 True
47 True
48 True
49 True
50 True
